### PyTorch官方參考範例
https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [8]:
import os
import glob
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torchvision
import time
from PIL import Image

def CCSN_images(dataset_dir):
    classfolder = glob.glob(os.path.join(dataset_dir,"*"),recursive=True)
    class_name  = [f.split(os.path.sep)[-1] for f in classfolder]
    img_labels  = []
    img_list    = []

    for class_id, f in enumerate(classfolder):
        files = glob.glob(os.path.join(f,"*.jpg"),recursive=True)
        img_labels.extend([class_id]*len(files))
        img_list.extend(files)

    img_labels = np.array(img_labels)
    img_list   = np.array(img_list,dtype=object)

    return img_list.reshape((-1,1)), img_labels, class_name

### 定義Dataset

In [9]:
class CCSNImageDataset(torch.utils.data.Dataset):
    def __init__(self, img_list, img_labels, transform=None, target_transform=None):
        self.transform        = transform
        self.target_transform = target_transform
        self.img_list         = img_list
        self.img_labels       = img_labels

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = self.img_list[idx,0]
        image    = Image.open(img_path)
        label    = self.img_labels[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label


### 設定CCSN資料集路徑

In [10]:
CCSNDataset_Path = "/kaggle/input/cirrus-cumulus-stratus-nimbus-ccsn-database/CCSN_v2"
img_list, img_labels, label_names = CCSN_images(CCSNDataset_Path)

CCSNDataset = CCSNImageDataset(img_list,img_labels)

### 範例:透過Dataset載入並顯示部分資料集

In [ ]:
N = len(CCSNDataset)

plt.figure(figsize=(16,8))

for i in range(16):
    plt.subplot(4,8,i+1)
    image, label = CCSNDataset[np.random.randint(0,N)]
    plt.imshow(image)
    plt.title(label_names[label])
    plt.axis(False)
plt.tight_layout()
plt.show()

### 使用自訂

In [13]:
import torchvision

In [14]:
import torch
import torch.nn as nn

class DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(DenseLayer, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv1 = nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(nn.functional.relu(self.bn1(x)))
        out = torch.cat([x, out], 1)
        return out

class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, num_layers):
        super(DenseBlock, self).__init__()
        layers = []
        for i in range(num_layers):
            layers.append(DenseLayer(in_channels + i * growth_rate, growth_rate))
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)

class TransitionLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TransitionLayer, self).__init__()
        self.bn = nn.BatchNorm2d(in_channels)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        out = self.conv(nn.functional.relu(self.bn(x)))
        out = self.pool(out)
        return out

class DenseNet(nn.Module):
    def __init__(self, growth_rate=32, num_classes=10):
        super(DenseNet, self).__init__()
        num_layers = 6  # Number of layers per dense block

        self.conv1 = nn.Conv2d(3, growth_rate*2, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(growth_rate*2)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.block1 = DenseBlock(growth_rate*2, growth_rate, num_layers)
        self.trans1 = TransitionLayer(growth_rate*2 + num_layers*growth_rate, growth_rate*2)

        self.block2 = DenseBlock(growth_rate*2, growth_rate, num_layers)
        self.trans2 = TransitionLayer(growth_rate*2 + num_layers*growth_rate, growth_rate*2)

        self.block3 = DenseBlock(growth_rate*2, growth_rate, num_layers)
        self.trans3 = TransitionLayer(growth_rate*2 + num_layers*growth_rate, growth_rate*2)

        self.block4 = DenseBlock(growth_rate*2, growth_rate, num_layers)

        self.bn2 = nn.BatchNorm2d(growth_rate*2 + num_layers*growth_rate)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(growth_rate*2 + num_layers*growth_rate, num_classes)

    def forward(self, x):
        out = self.pool1(nn.functional.relu(self.bn1(self.conv1(x))))
        out = self.trans1(self.block1(out))
        out = self.trans2(self.block2(out))
        out = self.trans3(self.block3(out))
        out = self.block4(out)
        out = self.avgpool(nn.functional.relu(self.bn2(out)))
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out


In [ ]:
from tempfile import TemporaryDirectory

def train_model(model, criterion, optimizer, dataloaders, dataset_sizes, num_epochs=25, patience = 0,scheduler = None):

    device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = model.to(device)

    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_loss = None
        best_acc  = 0
        history = {'loss':[],'accuracy':[],'val_loss':[],'val_accuracy':[]}
        patience_c = 0
        for epoch in range(num_epochs):
            print(f'Epoch {epoch:3d}/{num_epochs - 1}',end=' ')

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                if phase == 'train' and scheduler:
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}',end=' ' if phase=='train' else '\n')
                if phase == 'train':
                    history['loss'].append(epoch_loss)
                    history['accuracy'].append(epoch_acc.cpu().numpy())

                # deep copy the model
                if phase == 'val':
                    history['val_loss'].append(epoch_loss)
                    history['val_accuracy'].append(epoch_acc.cpu().numpy())
                    if best_acc < epoch_acc.cpu().numpy():
                        best_acc = epoch_acc.cpu().numpy()
                    if epoch == 0 or epoch_loss < best_loss:
                        best_loss = epoch_loss
                        patience_c= 0
                        torch.save(model.state_dict(), best_model_params_path)
                    else:
                        patience_c += 1

            if patience_c > patience:
                break

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model, history

In [ ]:
def evaluate(model, dataloader):

    device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model.eval()   # Set model to evaluate mode

    running_loss = 0.0
    running_corrects = 0
    dataset_size     = 0
        # Iterate over data.
    for inputs, labels in dataloader:

        dataset_size += inputs.size(0)

        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / dataset_size
    epoch_acc = running_corrects.double() / dataset_size

    print(f'test Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return epoch_loss, epoch_acc.cpu().numpy()

In [ ]:
def build_model(class_number, trainable=True):
    densenet = DenseNet(num_classes=class_number)

    for param in densenet.parameters():
        param.requires_grad = trainable

    return densenet

In [ ]:
model = DenseNet(num_classes=len(label_names))
print(model)

In [ ]:
from sklearn.model_selection import StratifiedKFold

device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print('gpu is available')
else:
    print('cpu only')

data_transforms = {
    'train': torchvision.transforms.Compose([
        torchvision.transforms.RandomResizedCrop(224),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': torchvision.transforms.Compose([
        torchvision.transforms.Resize(256),
        torchvision.transforms.CenterCrop(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

skf        = StratifiedKFold(n_splits=5, shuffle=True, random_state=0) # 80 % for training and validation sets, 20 % for the test set

batch_size = 64
epochs     = 80
patience   = epochs//10

test_accuracy = []
test_loss     = []
histories     = []

criterion = nn.CrossEntropyLoss()


for i, (train_idx, test_idx) in enumerate(skf.split(img_list,img_labels)):

    print('-'*50)

    X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(img_list[train_idx],img_labels[train_idx],test_size=0.2,random_state=i)

    testloader = torch.utils.data.DataLoader(CCSNImageDataset(img_list[test_idx],img_labels[test_idx],data_transforms['train'],lambda x : torch.tensor(x,dtype=torch.long)),batch_size=1,shuffle=True)
    trainloader= torch.utils.data.DataLoader(CCSNImageDataset(X_train,y_train,data_transforms['train'],lambda x : torch.tensor(x,dtype=torch.long)),batch_size=batch_size,shuffle=True)
    validloader= torch.utils.data.DataLoader(CCSNImageDataset(X_valid,y_valid,data_transforms['val'],lambda x : torch.tensor(x,dtype=torch.long)),batch_size=batch_size,shuffle=True)

    modelx = build_model(len(label_names),True)
    modelx = modelx.to(device)
    # Observe that all parameters are being optimized
    optimizer_ft = optim.AdamW(modelx.parameters())
    trained_model, history = train_model(modelx, criterion, optimizer_ft,
                                         dataloaders={'train':trainloader,'val':validloader},
                                         dataset_sizes={'train':X_train.shape[0],'val':X_valid.shape[0]},
                                         patience=patience,
                                         num_epochs=epochs,
                                         scheduler = None)

    histories.append(history)
    print(f'Fold {i+1:2d}',end=' ')
    loss, acc = evaluate(trained_model, testloader)

    test_loss.append(loss)
    test_accuracy.append(acc)

### 顯示結果

In [ ]:
plt.figure(figsize=(8,3*len(histories)))

max_loss = 0
max_acc  = 0
for i,history in enumerate(histories):
    max_loss = max(max_loss,np.max(history['loss']),np.max(history['val_loss']))

max_loss *= 1.05
for i,history in enumerate(histories):
    plt.subplot(len(histories),2,i*2+1)
    plt.title(f'fold:{i+1}')
    plt.plot(history['loss'],label='train')
    plt.plot(history['val_loss'],label='valid')
    plt.xlabel('epoch')
    step = int(np.ceil(len(history['loss'])/5))
    plt.xticks(np.arange(0,len(history['loss']),step),[str(u+1) for u in np.arange(0,len(history['loss']),step)])
    plt.ylabel('loss')
    plt.ylim([0,max_loss])
    plt.grid(True)
    plt.legend()

    plt.subplot(len(histories),2,i*2+2)
    plt.title(f'fold:{i+1}')
    plt.plot(history['accuracy'],label='train')
    plt.plot(history['val_accuracy'],label='valid')
    plt.xticks(np.arange(0,len(history['accuracy']),step),[str(u+1) for u in np.arange(0,len(history['accuracy']),step)])
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.ylim([0,1.0])
    plt.grid(True)
    plt.legend()

plt.tight_layout()
plt.show()

plt.figure()
plt.bar(x=np.arange(len(test_accuracy)),height=np.array(test_accuracy))
plt.xlabel('fold')
plt.xticks(np.arange(len(test_accuracy)),[str(i+1) for i in np.arange(len(test_accuracy))])
plt.ylabel('accuracy')
plt.title(f'average accuracy rate:{np.mean(np.array(test_accuracy)):.3f}+/-{np.std(np.array(test_accuracy)):.3f}')
plt.grid(True)
plt.ylim([0,1.0])
plt.show()
print(f'average accuracy rate:{np.mean(np.array(test_accuracy)):.3f}+/-{np.std(np.array(test_accuracy)):.3f}')